In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from pathlib import Path

In [6]:
data_path = Path('..') / 'ALOHA' / 'Receiver' / 'DATA'

In [7]:
list(data_path.iterdir())

[PosixPath('../ALOHA/Receiver/DATA/wSen_50Hz_ALOHA'),
 PosixPath('../ALOHA/Receiver/DATA/noSen_100Hz_noALOHA'),
 PosixPath('../ALOHA/Receiver/DATA/wSen_50Hz_noALOHA'),
 PosixPath('../ALOHA/Receiver/DATA/noSen_50Hz_ALOHA'),
 PosixPath('../ALOHA/Receiver/DATA/noSen_50Hz_noALOHA'),
 PosixPath('../ALOHA/Receiver/DATA/noSen_10Hz_noALOHA')]